# Actor-Critic

REINFORCE에서와 같이 **episodic reward**를 reward function으로 설정하자. 정책 $\pi_\theta$에서 reward function $J(\theta)$는 다음과 같이 정의된다.

$$ J(\theta) = \mathbb{E}_{S_0 \sim d_0, \pi_\theta}\left[ \sum_{t=0}^{\infty}{\gamma^t R_{t+1}} \right] = \mathbb{E}_{S_0 \sim d_0, \pi_\theta}\left[ G_0 \right] = \mathbb{E}_{S_0 \sim d_0}\left[ v_{\pi_\theta} (S_0) \right] $$

Policy Gradient Theorem(episodic)에 의하여, 위 식의 그래디언트인 $\nabla_\theta J(\theta)$ 는 다음과 같다.

$$ \nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} \left[ \left. \sum_{t=0}^{T}{\gamma^t q_{\pi_\theta}(S_t, A_t) \nabla_\theta \log \pi_\theta (A_t \mid S_t)} \right| S_0 \sim d_0 \right]$$

또한, 수학적으로 다음과 같은 성질이 성립한다.

$$ \mathbb{E}_{\pi_\theta} \left[ b \cdot \nabla_\theta \log \pi_\theta (A_t \mid S_t) \right] = 0 $$

이 성질을 사용하면 단순히 $q_{\pi_\theta}(S_t, A_t)$ 를 reward로 사용하는 것이 아닌, 평균을 빼서 더 안정적인 reward를 계산할 수 있다. 그 평균값은 $v_{\pi_\theta}(S_t)$ 이며, 이를 반영한 gradient 식은 다음과 같다.

$$ \nabla_\theta J(\theta) = \mathbb{E}_{\pi_\theta} \left[ \left. \sum_{t=0}^{T}{\gamma^t (q_{\pi_\theta}(S_t, A_t) - v_{\pi_\theta}(S_t) ) \nabla_\theta \log \pi_\theta (A_t \mid S_t)} \right| S_0 \sim d_0 \right]$$

$q_{\pi_\theta}(S_t, A_t) = \mathbb{E}_{\pi_\theta}\left[ \left. G_t \right| S_t, A_t \right]$ 이므로 지금과 같은 stochastic 조건 하에서 $q_{\pi_\theta}(S_t, A_t) \approx G_t$ 로 간주할 수 있다. 다만 이 경우에서는 $v_{\pi_\theta}$ 함수를 사용할 수 있으므로 각 시행마다 발생하는 variance를 줄이기 위해 $G_t$ 대신 $G_t \approx R_{t+1} + \gamma v_{\textbf{w}}(S_{t+1})$ 를 사용한다. 또한 일반적으로 $\gamma^t$ 항은 생략한다.

기존의 Monte-Carlo식의 REINFORCE와는 다르게, Actor-Critic에서 업데이트는 매 $t$ 마다 아래처럼 이루어진다.

$$ \delta_t = R_{t+1} + \gamma v_{\textbf{w}}(S_{t+1}) - v_{\textbf{w}}(S_t) $$
$$ \textbf{w} \leftarrow \textbf{w} + \beta \cdot \delta_t \nabla_\textbf{w} v_{\textbf{w}}(S_t) $$
$$ \theta \leftarrow \theta + \alpha \cdot \delta_t \nabla_\theta \log \pi_\theta (A_t \mid S_t) $$

### 패키지 import
의존성 있는 패키지인 ``` numpy ```, ``` tensorflow ```, ``` keras ```를 import해야 한다.  
  
참고: keras는 단독으로 import하지 않고 반드시 ``` import tensorflow.keras as keras ```와 같이 ``` tensorflow ```를 통해 import해야 잠재적인 오류를 예방할 수 있다.  

주어진 모듈을 설치하기 위해서는 `pip install numpy`, `pip install tensorflow`를 실행하자.

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

### Actor & Critic Model Building
* ``` l_rate ```는 학습률을 나타낸다.
* ``` n_actions ```는 tabular action의 개수를 나타낸다.
* ``` input_dims ```는 state의 feature dimension이다.

In [2]:
class ActorNetwork(keras.Model):
    # pi 함수
    def __init__(self, l_rate, n_actions, input_dims):
        super(ActorNetwork, self).__init__()
        self.l_rate     = l_rate
        self.n_actions  = n_actions
        self.input_dims = input_dims

        self.layer1 = keras.layers.Dense(64, activation='relu')
        self.layer2 = keras.layers.Dense(64, activation='relu')
        self.layer3 = keras.layers.Dense(n_actions, activation='softmax')

        self.compile(optimizer=keras.optimizers.Adam(learning_rate=self.l_rate))

    def call(self, inputs):
        mid1 = self.layer1(inputs)
        mid2 = self.layer2(mid1)
        return self.layer3(mid2)

class CriticNetwork(keras.Model):
    # v 함수
    def __init__(self, l_rate, n_actions, input_dims):
        super(CriticNetwork, self).__init__()
        self.l_rate     = l_rate
        self.n_actions  = n_actions
        self.input_dims = input_dims

        self.layer1 = keras.layers.Dense(64, activation='relu')
        self.layer2 = keras.layers.Dense(64, activation='relu')
        self.layer3 = keras.layers.Dense(n_actions)

        self.compile(optimizer=keras.optimizers.Adam(learning_rate=self.l_rate))

    def call(self, inputs):
        mid1 = self.layer1(inputs)
        mid2 = self.layer2(mid1)
        return self.layer3(mid2)

In [3]:
def categoricalSelect(L):
    ''' L[i] 확률로 i를 반환하는 함수. 어떠한 i에도 해당하지 않으면 0 반환. 0 <= sum(L) <= 1인 경우에만 정의됨.'''
    L = list(tf.squeeze(L))
    pick = np.random.random()
    cumul = 0

    for i, pi in enumerate(L):
        if (cumul <= pick < cumul + float(pi)):
            return i
        cumul += pi

    return 0

class ActorCriticAgent:
    def __init__(self, input_dims, actor_rate, critic_rate, gamma, n_actions, filename):
        self.read_only = False

        self.gamma = gamma
        self.action = [i for i in range(n_actions)]

        self.policy = ActorNetwork(actor_rate, n_actions, input_dims)
        self.value  = CriticNetwork(critic_rate, n_actions, input_dims)

        # model 저장용
        self.filename = filename

    def choose_action(self, obs):
        ''' 내부 stochastic policy인 `self.policy`를 토대로 action 결정 '''
        state = np.array([obs], dtype=np.float32) # 관찰 결과를 numpy 스타일로 변환 (tf.convert_to_tensor도 ok)
        pr = self.policy(state) # 신경망으로 전달, 확률 pr에 저장
        return categoricalSelect(pr)

    def learn(self, state, action, reward, next_state, done):
        if self.read_only:
            raise Exception("ActorCriticDeploy class can only use model, not learn.")
            return
        
        state = tf.convert_to_tensor([state], dtype=tf.float32)
        next_state = tf.convert_to_tensor([next_state], dtype=tf.float32)

        with tf.GradientTape(persistent=True) as tape:  # Gradient를 policy/value에 2회 적용하므로 persistent
            Pr    = tf.squeeze(self.policy(state))
            Vthis = tf.squeeze(self.value(state))
            Vnext = tf.squeeze(self.value(next_state))

            LogPr = tf.math.log(Pr[action]) # pi(a|s)를 추적해 gradient를 씌워야 함
            Delta = reward + self.gamma * Vnext * (1 - int(done)) - Vthis

            ActorLoss  = - LogPr * Delta    # Reinforce와 같은 원리
            CriticLoss = Delta ** 2 # DeepMind 7강 참조. 결론부터 말하면 Loss는 RMS(v_pi(s) - v_w(s)) = RMS(G_t - v_w(s)) = RMS(delta)

        gradActor  = tape.gradient(ActorLoss, self.policy.trainable_variables)
        gradCritic = tape.gradient(CriticLoss, self.value.trainable_variables)
        del tape    # 사용이 끝난 tape는 GarbageCollector에게 전달

        self.policy.optimizer.apply_gradients(zip(gradActor, self.policy.trainable_variables))
        self.value .optimizer.apply_gradients(zip(gradCritic, self.value.trainable_variables))

    def save_model(self):
        if self.read_only:
            raise Exception("ActorCriticDeploy class can only use model, not save.")
            return

        self.value.save(self.filename + "-value")
        self.policy.save(self.filename + "-policy")
    
    def load_model(self):
        self.value = keras.models.load_model(self.filename + "-value")
        self.policy = keras.models.load_model(self.filename + "-policy")

class ActorCriticDeploy(ActorCriticAgent):
    ''' Model의 학습과 저장은 불가능하고 오직 사용만을 위한 class '''
    def __init__(self, filename):
        self.filename = filename
        self.read_only = True
        self.load_model()

## Model Evaluating

### Gym을 사용한 가상환경 준비
OpenAI에서 제공하는 gym 모듈을 사용해 보자. 이 모듈은 `pip install gym`을 통해 설치할 수 있다.

In [6]:
import gym
import matplotlib.pyplot as plt
from tqdm import tqdm

env = gym.make('CartPole-v1')
observation = env.reset()

### Agent 학습
아래의 코드 블록을 실행하면 랜덤하게 초기화된 cartAgent가 강화학습을 시작한다.

In [8]:
cartAgent = ActorCriticAgent(
    input_dims=env.observation_space.shape, 
    actor_rate=0.003, 
    critic_rate=0.003, 
    gamma=0.98,
    n_actions=env.action_space.n, 
    filename='cartpole-ac'
)

scores, scores_avg = list(), list()
prgress = tqdm(range(200))

for episode in prgress:
    # 환경 및 점수 초기화
    score = 0
    obs_this = env.reset()
    done = False

    while not done:
        # Agent가 선택
        action = cartAgent.choose_action(obs_this)
        obs_next, reward, done, info = env.step(action)
        cartAgent.learn(obs_this, action, reward, obs_next, done)

        # 보상 저장
        score += reward
        obs_this = obs_next
        # env.render() # 렉 걸릴 경우 제외

    scores.append(score) # 점수 기록
    scores_avg.append(np.mean(scores[-30:])) # 이동평균 산출
    prgress.set_description("score {:>3.1f} | recent average score {:>3.1f}"
        .format(scores[-1], scores_avg[-1]))

    if episode % 200 == 0 and episode > 0: # 100 episode마다 모델을 저장
        cartAgent.save_model()

plt.plot(np.arange(len(scores_avg)), np.array(scores_avg)) # 이동평균 점수 그래프 그리기

score 26.0 | recent average score 72.8: 100%|██████████| 200/200 [02:51<00:00,  1.17it/s]  


다음은 학습한 모델을 불러와 실행하는 블록이다.

In [ ]:
cartDeploy = ActorCriticDeploy('cartpole-ac')

scores, scores_avg = list(), list()
prgress = tqdm(range(5))

for episode in prgress:
    # 환경 및 점수 초기화
    score = 0
    obs_this = env.reset()
    done = False

    while not done:
        # Agent가 선택
        action = cartDeploy.choose_action(obs_this)
        obs_next, reward, done, info = env.step(action)

        # 보상 저장
        score += reward
        obs_this = obs_next
        env.render() # 렉 걸릴 경우 제외

    scores.append(score) # 점수 기록
    scores_avg.append(np.mean(scores[-30:])) # 이동평균 산출
    prgress.set_description("score {:>3.1f} | recent average score {:>3.1f} | epsilon {:>.2f}"
        .format(scores[-1], scores_avg[-1], cartDeploy.eps))

plt.plot(np.arange(len(scores)), np.array(scores)) # 이동평균 점수 그래프 그리기